In [1]:
#Import der Libraries

%matplotlib inline

import sys
print('Python version:', sys.version)

import IPython
print('IPython:', IPython.__version__)

import itertools

import numpy as np
print('numpy:', np.__version__)

import scipy
print('scipy:', scipy.__version__)

import pandas as pd
print('pandas:', pd.__version__)

import pandas_profiling

import matplotlib.pyplot as plt
#print('matplotlib:', plt.__version__)

import sklearn
print('scikit-learn:', sklearn.__version__)

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split

import seaborn as sns
print('seaborn', sns.__version__)

import nltk

import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
#from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

#nltk.download()

import string

Python version: 3.6.3 |Anaconda custom (64-bit)| (default, Oct 15 2017, 03:27:45) [MSC v.1900 64 bit (AMD64)]
IPython: 6.1.0
numpy: 1.13.3
scipy: 0.19.1
pandas: 0.20.3
scikit-learn: 0.19.1


D:\Software\Anaconda\lib\site-packages\pandas_profiling\base.py:20: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "D:\Software\Anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\Software\Anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\Software\Anaconda\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\Software\Anaconda\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "D:\Software\Anaconda\lib

seaborn 0.8.0


# Business Understanding

## *Aus der Kaggle Competition:*
*Discussing things you care about can be difficult. The threat of abuse and harassment online means that many people stop expressing themselves and give up on seeking different opinions. Platforms struggle to effectively facilitate conversations, leading many communities to limit or completely shut down user comments.*

# Data Understanding

## Daten einlesen

In [2]:
data_train = pd.read_csv('D:/Projekte/Toxicity NLP/Data/train.csv')
data_test = pd.read_csv('D:/Projekte/Toxicity NLP/Data/test.csv')

In [3]:
pandas_profiling.ProfileReport(data_train)

Es ist auffällig, dass der Anteil an nicht-toxischen Kommentaren, wesentlich größer ist (90,4%) als der Anteil an toxischen Kommentaren. Dies kann später zu einem Problem werden, da der Trainingsalgorithmus ein Bias bezüglich nicht-toxischer Kommentare ausbilden kann.

# Data Preparation

In [4]:
# Definiere Features (X) und Labels (y)
X = data_train['comment_text']#[:20000]
y = data_train['toxic']#[:20000]
print(X.shape)
print(y.shape)

(159571,)
(159571,)


In [5]:
# Split the Training-Set in two Parts for Training and Testing (75% | 25%), shuffle with random seed 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

### Vectorizing the dataset

In [6]:
# Instantiate vectorizer
vect = CountVectorizer(stop_words='english')

In [7]:
# Learn trainign data vocabulary, use it to create document-term matrix
%time vect.fit(X_train)
%time X_train_dtm = vect.transform(X_train)

Wall time: 6.57 s
Wall time: 5.66 s


In [8]:
# examine the document-term matrix
X_train_dtm

<119678x158761 sparse matrix of type '<class 'numpy.int64'>'
	with 3048882 stored elements in Compressed Sparse Row format>

In [9]:
# Transform testing data (using the fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(X_test)
X_test_dtm

<39893x158761 sparse matrix of type '<class 'numpy.int64'>'
	with 987705 stored elements in Compressed Sparse Row format>

# Modeling - Naive Bayes Ansatz

In [10]:
# import and instantiate Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [11]:
# train the model using X_train_dtm
%time nb.fit(X_train_dtm, y_train)

Wall time: 46.9 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [12]:
# make class prediction for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

# Evaluation - Naive Bayes

In [13]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.94964028776978415

Dieses Ergebnis ist aufgrund der hohen Unausgewogenheit der Daten kritisch zu bewerten. Würde der Klassifizierer nämlich stupdide jedes Kommentar als nicht-toxisch bewerten, würde trotzdem eine Accuracy von 90,4% erzielt werden (Null-Accuracy). Nähere Aufschlüsse bietet die Confusion-Matrix.

In [14]:
# calculate the log-loss
metrics.log_loss(y_test, y_pred_class)

1.7393722204139304

Log-Loss ist vermutlich sehr hoch, da der Benchmark falsche Vorhersagen (mit hoher Wahrscheinlichkeit) stark bestraft. Bei NB sind die Wahrscheinlichkeiten aber meist extrem ausgeprägt, also nahe bei 0 oder nahe bei 1.

In [15]:
# print the confusion matrix

metrics.confusion_matrix(y_test, y_pred_class)

array([[35528,   468],
       [ 1541,  2356]], dtype=int64)

37.884 Datensätze wurden korrekt vorhergesagt, 468 nicht-toxische Kommentare wurden als toxic eingestuft, 1541 toxische Kommentare wurden als nicht-toxisch eingestuft. Dieses unausgewogene Verhältnis zwischen dem Fehler erster Art und Fehler zweiter Art zeigt den Bias des Klassifizierers, ein Kommentar öfter als nicht-toxisch einzustufen, da damit tendentiell ein besseres Ergebnis erzielt wird. Die eigentliche Accuracy ist damit stark zu hinterfragen, da sie nur knapp 5% besser ist, als die Null-Accuracy.

# Modeling - Logistic Regression

In [16]:
# import and instantiate Logistic Regression model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [17]:
# train the model using X-train_dtm
%time logreg.fit(X_train_dtm, y_train)

Wall time: 13 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

# Evaluation - Logistic Regression

In [18]:
# make class predictions
y_pred_class = logreg.predict(X_test_dtm)

In [19]:
# calculate accuracy
metrics.accuracy_score(y_test, y_pred_class)

0.9510941769232697

In [20]:
# print the confusion matrix

metrics.confusion_matrix(y_test, y_pred_class)

array([[35714,   282],
       [ 1669,  2228]], dtype=int64)

37.942 Datensätze wurden korrekt vorhergesagt. Das sind unwesentlich mehr als beim NB-Verfahren. Weiterhin wurden 282 nicht-toxische Kommentare als toxic eingestuft und 1669 toxische Kommentare als nicht-toxisch eingestuft. Die Accuracy ist damit zwar leicht besser als beim NB-Verfahren, das Verhältnis von Fehler 1. und 2. Art daüfr aber nochmal deutlich schlechter. Insgesamt ist die logistische Regression für diesen Anwendungsfall damit sogar als weniger robust anzunehmen, als das NB-Verfahren.